In [4]:
import os
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import load_img, img_to_array

In [5]:
BASE_PATH = 'EchoScribe ASL Dataset'

In [6]:
labels = []
data = []

In [7]:
for folder in os.listdir(BASE_PATH):
    folder_path = os.path.join(BASE_PATH, folder)
    if os.path.isdir(folder_path):
        print(f"Currently processing folder: {folder}")
        for file_name in os.listdir(folder_path):
            if file_name.endswith('.jpg') or file_name.endswith('.jpeg'):
                image_path = os.path.join(folder_path, file_name)
                image = load_img(image_path, target_size=(128, 128))
                image_arr = img_to_array(image) / 255.0
                data.append(image_arr)
                labels.append(folder)

Currently processing folder: E
Currently processing folder: H


KeyboardInterrupt: 

In [ ]:
data = np.array(data)
labels = np.array(labels)

In [ ]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
labels = le.fit_transform(labels)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, labels, test_size=0.2, random_state=42)

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense

In [ ]:
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D(2, 2),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),
    Flatten(),
    Dense(512, activation='relu'),
    Dense(5, activation='softmax') 
])

In [ ]:
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [ ]:
history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_test, y_test))

In [ ]:
test_loss, test_acc = model.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_acc * 100:.2f}%")

In [ ]:
import pickle

In [ ]:
model_pkl_file = "model.pkl"
with open(model_pkl_file, 'wb') as file:
    pickle.dump(model, file)